## Fake News Classifier Using Bidirectional LSTM

Dataset: https://www.kaggle.com/c/fake-news/data#

In [1]:
import pandas as pd
#!pip install kaggle



In [7]:
!pip install gensim==4.3.1
!pip install transformers
!pip install tensorflow

In [3]:
import gensim.downloader as api
wv = api.load('word2vec-google-news-300')

In [9]:
from transformers import BertTokenizer, TFBertModel
import torch

RuntimeError: Failed to import transformers.models.bert.modeling_tf_bert because of the following error (look up to see its traceback):
DLL load failed while importing _errors: The specified procedure could not be found.

In [ ]:
model_name = 'bert-base-uncased'  
tokenizer = BertTokenizer.from_pretrained(model_name)
bert_model = TFBertModel.from_pretrained(model_name)

!kaggle competitions download -c fake-news

In [4]:
import os
os.getcwd()

'C:\\Users\\u013709\\FakeNews\\Fake-New-LSTM'

In [6]:
df=pd.read_csv(r'C:\Users\u013709\Kaggle\FakeNews\train.csv')

In [7]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [9]:
###Drop Nan Values
df=df.dropna()


In [10]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [11]:
## Get the Dependent features
y=df['label']

In [12]:
y.value_counts()

label
0    10361
1     7924
Name: count, dtype: int64

In [13]:
X.shape

(18285, 4)

In [14]:
y.shape

(18285,)

In [15]:
import tensorflow as tf

In [16]:
tf.__version__

'2.16.1'

In [17]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [18]:
### Vocabulary size
voc_size=5000

### Onehot Representation

In [19]:
messages=X.copy()

In [20]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [21]:
messages.reset_index(inplace=True)

In [22]:
import nltk
import re
from nltk.corpus import stopwords

In [23]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\u013709\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [24]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')  # Download WordNet (lexical database)
nltk.download('omw-1.4')
nltk.download('punkt')  # For sentence tokenization
nltk.download('averaged_perceptron_tagger')  # For POS tagging
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\u013709\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\u013709\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\u013709\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\u013709\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [25]:

lemmatizer = WordNetLemmatizer()

#take mapping
def get_wordnet_pos(treebank_tag):
    """Maps treebank POS tags to WordNet POS tags."""
    if treebank_tag.startswith('J'):
        return 'a'  # Adjective
    elif treebank_tag.startswith('V'):
        return 'v'  # Verb
    elif treebank_tag.startswith('N'):
        return 'n'  # Noun
    elif treebank_tag.startswith('R'):
        return 'r'  # Adverb
    else:
        return ''

def lemmatiser_word(sentence):
    lemma =[]
    tokens = word_tokenize(sentence)  # Tokenize the sentence into words
    tagged_words = pos_tag(tokens)    # Tag each word with its POS
    for word,tag in tagged_words:
        if not word in stopwords.words('english'):
            pos_tag_simple=get_wordnet_pos(tag)
            if pos_tag_simple:
                lemma.append(lemmatizer.lemmatize(word, pos=pos_tag_simple))
            else:
                lemma.append(lemmatizer.lemmatize(word))
            
    return " ".join(lemma)

In [43]:
print(lemmatiser_word(messages['title'][20]))

Abortion Pill Orders Rise 7 Latin American Nations Zika Alert - The New York Times


In [44]:
### Dataset Preprocessingfrom nltk.stem import WordNetLemmatizer


from nltk.stem.porter import PorterStemmer

ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    #print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    #review = review.split()
    
    #review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    #review = ' '.join(review)
    review = lemmatiser_word(review)
    
    corpus.append(review)

In [47]:
corpus

['house dem aide even see comey letter jason chaffetz tweet',
 'flynn hillary clinton big woman campus breitbart',
 'truth might get fire',
 'civilian kill single u airstrike identify',
 'iranian woman jail fictional unpublished story woman stone death adultery',
 'jackie mason hollywood would love trump bomb north korea lack trans bathroom exclusive video breitbart',
 'beno hamon win french socialist party presidential nomination new york time',
 'back channel plan ukraine russia courtesy trump associate new york time',
 'obama organize action partner soros link indivisible disrupt trump agenda',
 'bbc comedy sketch real housewife isi cause outrage',
 'russian researcher discover secret nazi military base treasure hunter arctic photo',
 'u official see link trump russia',
 'yes paid government troll social medium blog forum website',
 'major league soccer argentine find home success new york time',
 'well fargo chief abruptly step new york time',
 'anonymous donor pay million release 

#onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr=encoding_tfidf(corpus)
pad_length=20

array=[]
for i in range(len(corpus)):
    array.append(drop_zeroes(onehot_repr[i]))
    
pad_sequences(array,padding='pre',maxlen=pad_length)    

array.shape

In [30]:
def drop_zeroes(array):
    pad_length = 20
    return_array=[]
    
    for i in array:
        if i == 0 :
            pass
        else :
            return_array.append(i)
    return return_array



In [31]:
print(drop_zeroes([0,0,0,0,0,0,2, 2 , 3 ,6,0,0,0,0,0,0,0]))

[2, 2, 3, 6]


In [48]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
feature_names=[]
def encoding_tfidf(corpus):
    # Vectorize the text data using TF-IDF
    global  voc_size
    global feature_names
    vectorizer = TfidfVectorizer()
    data = vectorizer.fit_transform(corpus)
    # Get the feature names (vocabulary)
    feature_names = vectorizer.get_feature_names_out()
    print()
    word_to_index = {word: index for index, word in enumerate(feature_names)}
    voc_size =len(feature_names)
    #Numerical Representation (Using TF-IDF Values)
    sequences = []
    for row in data.toarray():
        sentence_indices = [word_to_index[feature_names[i]] for i in np.nonzero(row)[0]]
        sequences.append(sentence_indices)
    return sequences


vectorizer = TfidfVectorizer(max_features= 300)
data = vectorizer.fit_transform(corpus)

#for j in range(data.toarray().shape[0]):
#    for i in range(data.toarray().shape[1]):
#        if data.toarray()[j][i] != 0 :
#            print(f' message {j}  word {i} = {data.toarray()[j][i]}')

In [95]:
from gensim.models import Word2Vec


In [96]:
def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    return [word for word in tokens if word not in stopwords.words('english') and word.isalnum()]

In [97]:
corpus_wv = [preprocess_text(message) for message in corpus]

In [98]:
corpus_wv

[['house',
  'dem',
  'aide',
  'even',
  'see',
  'comey',
  'letter',
  'jason',
  'chaffetz',
  'tweet'],
 ['flynn', 'hillary', 'clinton', 'big', 'woman', 'campus', 'breitbart'],
 ['truth', 'might', 'get', 'fire'],
 ['civilian', 'kill', 'single', 'u', 'airstrike', 'identify'],
 ['iranian',
  'woman',
  'jail',
  'fictional',
  'unpublished',
  'story',
  'woman',
  'stone',
  'death',
  'adultery'],
 ['jackie',
  'mason',
  'hollywood',
  'would',
  'love',
  'trump',
  'bomb',
  'north',
  'korea',
  'lack',
  'trans',
  'bathroom',
  'exclusive',
  'video',
  'breitbart'],
 ['beno',
  'hamon',
  'win',
  'french',
  'socialist',
  'party',
  'presidential',
  'nomination',
  'new',
  'york',
  'time'],
 ['back',
  'channel',
  'plan',
  'ukraine',
  'russia',
  'courtesy',
  'trump',
  'associate',
  'new',
  'york',
  'time'],
 ['obama',
  'organize',
  'action',
  'partner',
  'soros',
  'link',
  'indivisible',
  'disrupt',
  'trump',
  'agenda'],
 ['bbc', 'comedy', 'sketch', '

In [101]:

model = Word2Vec(sentences=corpus_wv, vector_size=300, window=5, min_count=1, workers=4)

In [102]:

embedding_matrix = np.zeros((len(model.wv), 300))
for i, word in enumerate(model.wv.key_to_index):
    embedding_matrix[i] = model.wv[word]

In [103]:
embedding_matrix.shape

(16086, 300)

In [104]:
vocab=model.wv.key_to_index
voc_size=embedding_matrix.shape[0]

In [105]:
word_to_index = {word: index for index, word in enumerate(vocab)}

In [106]:
word_to_index

{'new': 0,
 'time': 1,
 'york': 2,
 'trump': 3,
 'breitbart': 4,
 'hillary': 5,
 'clinton': 6,
 'u': 7,
 'donald': 8,
 'say': 9,
 'election': 10,
 'obama': 11,
 'state': 12,
 'video': 13,
 'war': 14,
 'russia': 15,
 'get': 16,
 'president': 17,
 'comment': 18,
 'world': 19,
 'year': 20,
 'attack': 21,
 'vote': 22,
 'day': 23,
 'news': 24,
 'call': 25,
 'american': 26,
 'report': 27,
 'email': 28,
 'fbi': 29,
 'go': 30,
 'america': 31,
 'make': 32,
 'house': 33,
 'white': 34,
 'woman': 35,
 'police': 36,
 'medium': 37,
 'kill': 38,
 'find': 39,
 'campaign': 40,
 'take': 41,
 'plan': 42,
 'bill': 43,
 'first': 44,
 'one': 45,
 'show': 46,
 'brief': 47,
 'win': 48,
 'black': 49,
 'syria': 50,
 'anti': 51,
 'break': 52,
 'right': 53,
 'back': 54,
 'watch': 55,
 'russian': 56,
 'man': 57,
 'people': 58,
 'big': 59,
 'may': 60,
 'use': 61,
 'life': 62,
 'come': 63,
 'democrat': 64,
 'million': 65,
 'muslim': 66,
 'could': 67,
 'want': 68,
 'like': 69,
 'ban': 70,
 'child': 71,
 'china': 72,


In [107]:


def text_to_sequence(text):
    return [word_to_index[word] for word in text]

sequences = [text_to_sequence(message) for message in corpus_wv]

In [108]:
sequences

[[33, 559, 588, 137, 87, 90, 490, 1772, 3276, 401],
 [813, 5, 6, 59, 35, 782, 4],
 [325, 834, 16, 85],
 [616, 38, 1099, 7, 1928, 1930],
 [1145, 35, 583, 4401, 9808, 258, 35, 1411, 127, 9847],
 [3727, 3708, 328, 155, 285, 3, 269, 122, 311, 1256, 2055, 1218, 241, 13, 4],
 [6711, 6991, 48, 371, 2351, 120, 157, 1216, 0, 2, 1],
 [54, 1337, 42, 1026, 15, 7002, 3, 1904, 0, 2, 1],
 [11, 4137, 555, 2047, 257, 327, 9452, 1983, 3, 821],
 [1159, 1875, 6101, 166, 6231, 102, 413, 1102],
 [56, 2223, 862, 220, 1162, 233, 685, 4659, 3261, 3237, 305],
 [7, 105, 87, 327, 3, 15],
 [1937, 7281, 95, 1213, 446, 37, 1470, 4546, 1057],
 [493, 1380, 1381, 5924, 39, 124, 1383, 0, 2, 1],
 [516, 1437, 189, 5973, 321, 0, 2, 1],
 [959, 717, 134, 65, 167, 952, 100, 376, 441, 307],
 [29, 253, 5],
 [1748, 1817, 2042, 8, 3, 175, 842, 4],
 [4462, 10082, 6, 199, 256, 212, 26, 226, 258],
 [2187, 10192, 3, 2027, 10274, 4],
 [605, 2091, 92, 201, 2643, 26, 275, 448, 829, 0, 2, 1],
 [2053, 164, 1435, 1751, 70, 210, 464],
 [241

In [109]:
max_length = 20
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='pre')
pd.DataFrame(padded_sequences,columns=[f'column{column}' for column in range(1,21)])

,column1,column2,column3,column4,column5,column6,column7,column8,column9,column10,column11,column12,column13,column14,column15,column16,column17,column18,column19,column20
0,0,0,0,0,0,0,0,0,0,0,33,559,588,137,87,90,490,1772,3276,401
1,0,0,0,0,0,0,0,0,0,0,0,0,0,813,5,6,59,35,782,4
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,325,834,16,85
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,616,38,1099,7,1928,1930
4,0,0,0,0,0,0,0,0,0,0,1145,35,583,4401,9808,258,35,1411,127,9847
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18280,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1822,3,2035,71,34,4744
18281,0,0,0,0,0,0,0,0,0,0,75,112,322,2895,2821,5230,3015,0,2,1
18282,0,0,0,0,0,0,0,0,0,0,6078,9,830,1824,1856,4745,3325,0,2,1
18283,0,0,0,0,0,0,0,0,0,0,0,0,0,0,469,15,314,4772,2394,5165


In [127]:
padded_sequences.shape

(18285, 20)

In [110]:
{word: index for index, word in enumerate(feature_names)}

{}

In [153]:
encoding_tfidf(corpus)

[[293, 2314, 2742, 3641, 4872, 6736, 7460, 8193, 12710, 14769],
 [1426, 1785, 2084, 2592, 5428, 6559, 15830],
 [5319, 5849, 9042, 14704],
 [318, 2523, 6863, 7787, 13091],
 [193, 3481, 5259, 7337, 7431, 13687, 13703, 15024, 15830],
 [1223,
  1631,
  1785,
  4924,
  6638,
  7418,
  7883,
  7951,
  8425,
  8750,
  9793,
  14553,
  14676,
  15312,
  15874],
 [1355, 5599, 6273, 9663, 9766, 10419, 11106, 13284, 14385, 15765, 15988],
 [857, 1052, 2340, 3116, 9663, 10788, 12366, 14385, 14676, 14822, 15988],
 [122, 259, 4051, 7067, 8293, 9901, 10115, 10416, 13358, 14676],
 [1250, 2243, 2738, 6740, 7364, 10195, 11616, 13126],
 [733, 1198, 3992, 6800, 9066, 9562, 10683, 11950, 12368, 12697, 14599],
 [8293, 9974, 12366, 12710, 14676],
 [1558, 5520, 6011, 8885, 10297, 13282, 14645, 15587, 15962],
 [742, 5305, 6648, 8094, 8569, 9663, 13280, 13817, 14385, 15988],
 [45, 2424, 5119, 9663, 13641, 14385, 15624, 15988],
 [76, 580, 778, 3374, 4174, 4883, 9076, 10479, 10758, 11828],
 [2601, 5162, 6559],
 [1

In [129]:
voc_size


16086

### Embedding Representation

In [155]:
sent_length=20
#embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
embedded_docs=pad_sequences(encoding_tfidf(corpus),padding='pre',maxlen=sent_length)
pd.DataFrame(embedded_docs,columns=[f'column{column}' for column in range(1,21)])

,column1,column2,column3,column4,column5,column6,column7,column8,column9,column10,column11,column12,column13,column14,column15,column16,column17,column18,column19,column20
0,0,0,0,0,0,0,0,0,0,0,293,2314,2742,3641,4872,6736,7460,8193,12710,14769
1,0,0,0,0,0,0,0,0,0,0,0,0,0,1426,1785,2084,2592,5428,6559,15830
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5319,5849,9042,14704
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,318,2523,6863,7787,13091
4,0,0,0,0,0,0,0,0,0,0,0,193,3481,5259,7337,7431,13687,13703,15024,15830
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18280,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2425,10983,11558,13922,14676,15683
18281,0,0,0,0,0,0,0,0,0,0,0,0,0,8773,9663,9954,10818,12577,14385,15988
18282,0,0,0,0,0,0,0,0,0,0,694,1241,6764,8514,9663,11653,12542,14095,14385,15988
18283,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1107,4938,6626,9544,10366,12366


In [82]:
pd.DataFrame(padded_sequences,columns=[f'column{column}' for column in range(1,21)])

,column1,column2,column3,column4,column5,column6,column7,column8,column9,column10,column11,column12,column13,column14,column15,column16,column17,column18,column19,column20
0,0,0,0,0,0,0,0,0,0,0,33,559,588,137,87,90,490,1772,3276,401
1,0,0,0,0,0,0,0,0,0,0,0,0,0,813,5,6,59,35,782,4
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,325,834,16,85
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,616,38,1099,7,1928,1930
4,0,0,0,0,0,0,0,0,0,0,1145,35,583,4401,9808,258,35,1411,127,9847
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18280,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1822,3,2035,71,34,4744
18281,0,0,0,0,0,0,0,0,0,0,75,112,322,2895,2821,5230,3015,0,2,1
18282,0,0,0,0,0,0,0,0,0,0,6078,9,830,1824,1856,4745,3325,0,2,1
18283,0,0,0,0,0,0,0,0,0,0,0,0,0,0,469,15,314,4772,2394,5165


In [130]:
embedding_layer = Embedding(input_dim=voc_size, 
                            output_dim=300, 
                            weights=[embedding_matrix], 
                            input_length=max_length, 
                            trainable=False)

In [131]:
## Creating model
embedding_vector_features=40
model=Sequential()
model1.add(embedding_layer)
#model.add(Embedding(voc_size,embedding_vector_features,input_length=max_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_4 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [132]:
## Creating model
sent_length=max_length
embedding_vector_features=40
model1=Sequential()
#model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(embedding_layer)
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.5))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)         │ (None, 1, 300)         │     4,825,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_2 (Bidirectional) │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,825,800 (18.41 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 4,825,800 (18.41 MB)

None


In [133]:
len(padded_sequences),y.shape

(18285, (18285,))

In [134]:
data=embedding_matrix

In [136]:
import numpy as np
#X_final=padded_sequences
X_final= padded_sequences
y_final=np.array(y)

In [137]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [138]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [139]:
### Finally Training
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 20s 70ms/step - accuracy: 0.8104 - loss: 0.4116 - val_accuracy: 0.8726 - val_loss: 0.2938
Epoch 2/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 12s 63ms/step - accuracy: 0.8770 - loss: 0.2944 - val_accuracy: 0.8837 - val_loss: 0.2711
Epoch 3/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 13s 67ms/step - accuracy: 0.8845 - loss: 0.2716 - val_accuracy: 0.8860 - val_loss: 0.2613
Epoch 4/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 12s 60ms/step - accuracy: 0.8871 - loss: 0.2629 - val_accuracy: 0.8845 - val_loss: 0.2499
Epoch 5/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 12s 63ms/step - accuracy: 0.8927 - loss: 0.2489 - val_accuracy: 0.8822 - val_loss: 0.2507
Epoch 6/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 13s 65ms/step - accuracy: 0.8931 - loss: 0.2479 - val_accuracy: 0.8775 - val_loss: 0.2586
Epoch 7/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 13s 65ms/step - accuracy: 0.8909 - loss: 0.2402 - val_accuracy: 0.8918 - val_loss: 0.2421
Epoch 8/10
192/192 ━━━━━━━━━━━━━━━━━━━━ 14s 70ms/step - accuracy: 0.8901 - loss: 0.2454 - 

### Performance Metrics And Accuracy

In [140]:

y_pred=model1.predict(X_test)
y_pred1=[1 if i>=0.5 else 0 for i in y_pred ]

189/189 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step


In [141]:
#y_pred1

In [142]:
from sklearn.metrics import confusion_matrix

In [143]:
confusion_matrix(y_test,y_pred1)

array([[2803,  616],
       [  32, 2584]], dtype=int64)

In [144]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred1)

0.8926263463131732

In [145]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred1))

              precision    recall  f1-score   support

           0       0.99      0.82      0.90      3419
           1       0.81      0.99      0.89      2616

    accuracy                           0.89      6035
   macro avg       0.90      0.90      0.89      6035
weighted avg       0.91      0.89      0.89      6035

